# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import pathlib

# Loading data

In [ ]:
# File paths

## Level 1 for rural/urban and multiplier values
level_01_path = '../../../Dta raw files/level_01.dta'

## Level 14 for category wise summary expenditure
level_14_path = '../../../Dta raw files/level_14.dta'

## Level 15 for hh size
level_15_path = '../../../Dta raw files/level_15.dta'

In [16]:
# Reading level 01 file

df_level_01 = pd.read_stata(level_01_path)
df_level_01 = df_level_01[['hhid', 'sector', 'multiplier']]
df_level_01 = df_level_01.sort_values(by = 'hhid')
df_level_01 = df_level_01.set_index('hhid')
print(f"Number of households in the file: {df_level_01.shape[0]}")
df_level_01.head()

Number of households in the file: 261953


,sector,multiplier
hhid,,
22300101,2,57436
22300201,2,27497
22300202,2,27497
22300203,2,27497
22300204,2,27497


In [20]:
# Reading level 14 file
df_level_14 = pd.read_stata(level_14_path)
print(f"Number of rows: {df_level_14.shape[0]}")
print("Each household data is spread across multiple rows")
df_level_14.head()

Number of rows: 8296569
Each household data is spread across multiple rows


,index,hhid,questionnaire_num,level,section,item_code,value,multiplier
0,0,22300101,C,10,12.1,309,20,57436
1,1,22300101,F,5,6.7,269,261,57436
2,2,22300101,F,5,6.4,249,130,57436
3,3,22300101,F,5,6.3,239,345,57436
4,4,22300101,C,9,9.1,459,1310,57436


In [34]:
# Reading level 15 file
df_level_15 = pd.read_stata(level_15_path)
df_level_15 = df_level_15.pivot(index = "hhid", columns="questionnaire_num", values = 'hh_size')
df_level_15 = df_level_15[['C','D','F']] #Preserving only FDQ questionnaire hh size
df_level_15.head()

questionnaire_num,C,D,F
hhid,,,
22300101,4.0,4.0,4.0
22300201,3.0,3.0,4.0
22300202,2.0,2.0,2.0
22300203,2.0,2.0,2.0
22300204,2.0,2.0,2.0


# Total expenditure

## Commodity mapping

In [36]:
commodity_mapping = {'food_w': {
                                'description': 'Food items with recorded weekly expenditure', 
                                'item_list': [169, 219, 239, 249, 199, 189, 269, 279, 289, 299],
                                'duration': 7
                                },
                     'food_m': {
                                'description': 'Food items with recorded monthly expenditure', 
                                'item_list': [129,139,159,179],
                                'duration': 30
                                },
                     'consumables_w': {
                                'description': 'Consumable items with recorded weekly expenditure', 
                                'item_list': [309,319,329],
                                'duration': 7
                                },
                     'consumables_m': {
                                'description': 'Consumable items with recorded monthly expenditure', 
                                'item_list': [349,459,479,429,519,499,439,529],
                                'duration': 30
                                },
                     'consumables_y': {
                                'description': 'Consumable items with recorded yearly expenditure', 
                                'item_list': [409,419,899],
                                'duration': 365
                                },
                     'durables_y': {
                                'description': 'Durable items with recorded yearly expenditure', 
                                'item_list': [379,399,389,629,609,99,619,599,579,559,569,639,649],
                                'duration': 365
                                },
                    }

In [50]:
# Check if all elements are covered in commodity mapping

## Items covered in commodity mapping
mapped_list = []
for key,dict in commodity_mapping.items():
    mapped_list = mapped_list + dict['item_list']

## Items in df but not in commodity mapping
print(set([item for item in df_level_14['item_code'] if item not in mapped_list]))
print('Itemcode 539 can be ignored as it corresponds to imputed rent. It is not considered while deriving total expenditure')

{539}
Itemcode 539 can be ignored as it corresponds to imputed rent. It is not considered while deriving total expenditure


## Scaling function

In [51]:
#Scaling function to convert expenditure based on hh_size (CSQ/DGQ) to FDQ

def scaling_column (hh_series, old_hsize_series, new_hsize_series):
    scaled_series = hh_series.div(old_hsize_series, axis = 'index')
    scaled_series = scaled_series.mul(new_hsize_series, axis = 'index')
    return scaled_series

# Aggregate expenditure function

In [57]:
#Method to merge weekly/monthly/yearly expenditure into total monthly expenditure for the category 

def expenditure (hh_series, commodity_mapping = commodity_mapping):
    series_set = set(hh_series.index)
    commodity_set = set(commodity_mapping.keys())

    if series_set.issubset(commodity_set):    
        durations = hh_series.index.map(lambda x: commodity_mapping[x]['duration'] ) #Creating a series of duration
        item_monthly = (hh_series/durations)*30
        item_monthly = item_monthly.round(2)
        total_exp = item_monthly.sum()
    else: 
        print("List not in commodity mapping")
    return total_exp

## Pivoting and scaling each expenditure in Level 14 to FDQ hh size

In [56]:
df_pivot = df_level_14.pivot(index = 'hhid', columns="item_code", values = 'value')
print(f"# of households = {df_pivot.shape[0]}")

# Scaling FDQ items
ic_food = [item for key in ['food_w', 'food_m'] for item in commodity_mapping [key]['item_list']] #List of FDQ items
df_pivot[ic_food] = df_pivot[ic_food].apply(lambda col: scaling_column(hh_series= col, 
                                                                       old_hsize_series= df_level_15['F'], 
                                                                       new_hsize_series= df_level_15['F']))

# Scaling CSQ items
ic_consumables = [item for key in ['consumables_w', 'consumables_m', 'consumables_y'] for item in commodity_mapping [key]['item_list']] #List of CSQ items
df_pivot[ic_consumables] = df_pivot[ic_consumables].apply(lambda col: scaling_column(hh_series= col, 
                                                                                     old_hsize_series= df_level_15['C'], 
                                                                                     new_hsize_series= df_level_15['F']))

# Scaling DGQ items
ic_durables = [item for key in ['durables_y'] for item in commodity_mapping [key]['item_list']] #List of DGQ items
df_pivot[ic_durables] = df_pivot[ic_durables].apply(lambda col: scaling_column(hh_series= col, 
                                                                                     old_hsize_series= df_level_15['D'], 
                                                                                     new_hsize_series= df_level_15['F']))

#Display
df_pivot = df_pivot.round(2)
df_pivot.head()

# of households = 261953


item_code,99,129,139,159,169,179,189,199,219,239,...,559,569,579,599,609,619,629,639,649,899
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,600.0,1687.0,NaN,562.0,380.0,125.0,259.0,440.0,336.0,345.0,...,1000.00,800.00,700.00,3450.00,12800.0,NaN,2700.00,2100.00,1000.0,1500.00
22300201,NaN,1665.0,NaN,470.0,270.0,125.0,209.0,592.0,394.0,395.0,...,266.67,1133.33,1666.67,4733.33,606000.0,NaN,5506.67,4666.67,800.0,3333.33
22300202,NaN,856.0,NaN,370.0,205.0,58.0,183.0,496.0,225.0,330.0,...,1200.00,750.00,610.00,3200.00,8600.0,NaN,4030.00,1800.00,400.0,1800.00
22300203,NaN,873.0,NaN,288.0,103.0,57.0,70.0,478.0,101.0,225.0,...,780.00,1200.00,550.00,2150.00,10500.0,NaN,4200.00,450.00,550.0,2300.00
22300204,NaN,1075.0,NaN,214.0,178.0,56.0,145.0,552.0,169.0,530.0,...,4200.00,NaN,1880.00,5300.00,10300.0,NaN,2980.00,2100.00,500.0,3000.00


## Total expenditure

In [59]:
#Generating total expenditure for each key in commodity mapping
for key in commodity_mapping.keys():
    df_pivot[key] = df_pivot[commodity_mapping[key]['item_list']].apply(lambda x: x.sum(), axis = 1)

In [61]:
#Generating total monthly expenditures for each of the categories: food, consumables, and durables
df_pivot['food_total'] = df_pivot[['food_w', 'food_m']].apply(expenditure, axis = 1)
df_pivot['consumables_total'] = df_pivot[['consumables_w', 'consumables_m', 'consumables_y']].apply(expenditure, axis = 1)
df_pivot['durables_total'] = df_pivot[['durables_y']].apply(expenditure, axis = 1) #df_pivot[['durables_y']] because the expenditure function is made for a dataframe

In [107]:
#Total monthly expenditure of HHs
df_pivot['total_expenditure'] = df_pivot['food_total'] + df_pivot['consumables_total'] + df_pivot['durables_total']

In [110]:
df_pivot.head()

item_code,99,129,139,159,169,179,189,199,219,239,...,food_w,food_m,consumables_w,consumables_m,consumables_y,durables_y,food_total,consumables_total,durables_total,total_expenditure
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,600.0,1687.0,NaN,562.0,380.0,125.0,259.0,440.0,336.0,345.0,...,2649.0,2374.0,960.0,16300.00,1500.00,40935.00,13726.86,20537.58,3364.52,37628.96
22300201,NaN,1665.0,NaN,470.0,270.0,125.0,209.0,592.0,394.0,395.0,...,3273.0,2260.0,0.0,30201.33,3333.33,667120.01,16287.14,30475.30,54831.78,101594.22
22300202,NaN,856.0,NaN,370.0,205.0,58.0,183.0,496.0,225.0,330.0,...,2071.0,1284.0,13.0,10326.00,1800.00,34500.00,10159.71,10529.66,2835.62,23524.99
22300203,NaN,873.0,NaN,288.0,103.0,57.0,70.0,478.0,101.0,225.0,...,1577.0,1218.0,211.0,11142.00,2300.00,35850.00,7976.57,12235.33,2946.58,23158.48
22300204,NaN,1075.0,NaN,214.0,178.0,56.0,145.0,552.0,169.0,530.0,...,2473.0,1345.0,0.0,13558.00,3000.00,60250.00,11943.57,13804.58,4952.05,30700.20


## Calculating MPCE + Assigning separate deciles to rural and urban sector

In [111]:
## Merging expenditure and hh_size table with df_pivot
df_mpce = pd.merge(df_pivot[['food_total', 'consumables_total', 'durables_total', 'total_expenditure']], df_level_15['F'], 
                   how = 'inner', 
                   #indicator = True,
                   left_index = True,
                   right_index = True)

## Merging df_mpce and level 01 (sector and multiplier)
df_mpce = pd.merge(df_mpce, df_level_01,
                     how = 'inner', 
                    #indicator = True,
                    left_index = True,
                    right_index = True)

## Renaming hh_size column
df_mpce = df_mpce.rename(columns = {"F": "hh_size"})

## Expanding each row hh_size number of times
df_mpce = df_mpce.loc[df_mpce.index.repeat(df_mpce['hh_size'])]

## MPCE for each household
df_mpce['mpce'] = round(df_mpce['total_expenditure']/df_mpce['hh_size'],2)

# Calculating rural-urban deciles
## Sorting by sector and MPCE for decile calculation
df_mpce = df_mpce.sort_values(by = ['sector','mpce'])

## Normalizing multiplier values to range between 0 and 100
df_mpce["normal_multiplier"] = df_mpce.groupby('sector')['multiplier'].transform(lambda x: x*(100/x.sum())) #groupby requires use of transform (and not apply)

## Cumulative normalised values
df_mpce['cum_normal_multiplier'] = df_mpce.groupby('sector')['normal_multiplier'].cumsum()

## Decile classes based on cumulative normalised multiplier values
df_mpce['decile'] = pd.cut(df_mpce['cum_normal_multiplier'],
                             bins = 10,
                             labels = range(1,11),
                             include_lowest=False
                             )

##Collapsing df_mpce back to only keep the first row
df_mpce = df_mpce.groupby("hhid").first()

# Weighted MPCE for each decile

In [112]:
# Calculating weighted MPCE by sector and decile
mult_exp = df_mpce.groupby(['sector', 'decile'], observed = False).apply(lambda table: (table['total_expenditure']*table['multiplier']).sum(), include_groups = False)
mult_hh_size = df_mpce.groupby(['sector', 'decile'], observed = False).apply(lambda table: (table['hh_size']*table['multiplier']).sum(), include_groups = False)
wt_mpce = (mult_exp/mult_hh_size).round(2)
display(wt_mpce)

sector  decile
1       1          1901.35
        2          2473.16
        3          2833.26
        4          3161.91
        5          3497.55
        6          3866.09
        7          4303.84
        8          4885.30
        9          5762.94
        10         8532.59
2       1          2734.42
        2          3687.38
        3          4353.32
        4          4978.62
        5          5621.67
        6          6333.96
        7          7199.28
        8          8352.51
        9         10138.71
        10        16563.35
dtype: float64

# Saving

In [ ]:
try:
    base_export_path = '../../../Python implementation/Data extraction/Population based MPCE/'

    # converting deciles to numeric (from categories)
    df_mpce['decile'] = df_mpce['decile'].astype(int)

#saving
    df_mpce.to_stata(base_export_path + "MPCE 2023_24 2Apr25.dta")
except Exception as e:
    print(e)